# Get started with Azure Cosmos DB development

Built-in Jupyter notebooks in Azure Cosmos DB allow to easily ingest and view your data - all within the Azure Portal. In this notebook, we'll learn how to create an Azure Cosmos container, ingest data, and query data.

To run this C# notebook, be sure to select the **CSharp** kernel in the command bar, so we can get all the language support features we need.


## Create database and containers
First, we'll create a new database and container to hold our data. Note when we create the container, we select a value for the partition key: we'll partition our data by the item id value, as it has a high cardinality (important for workloads during a lot of writes, e.g. IOT workloads) and evenly distributes the request and storage volume. Choosing a good partition key is "key" to getting good scale and performance from Azure Cosmos DB, so it's important we follow the [best practices](https://docs.microsoft.com/azure/cosmos-db/partitioning-overview)!

### The type of container this will create depends on the capacity mode of your Cosmos DB account:
- On a provisioned throughput account, this will create a container provisioned at 400 RU/s, the minimum RU/s for any provisioned throughput container.
- On a serverless account, this will create a serverless container.

In [1]:
using System;
using Microsoft.Azure.Cosmos;
using System.Collections;
using System.Collections.Generic;

// Initialize a new instance of CosmosClient using the built-in account endpoint and key parameters
CosmosClient cosmosClient = new CosmosClient(Cosmos.Endpoint, Cosmos.Key);

// Create a new database and a new container
Microsoft.Azure.Cosmos.Database database = await cosmosClient.CreateDatabaseIfNotExistsAsync("WebsiteData");

ContainerProperties containerProperties = new ContainerProperties()
{
    Id = "sales",
    PartitionKeyPath = "/id",
};

Container container = await database.CreateContainerIfNotExistsAsync(containerProperties, throughput: 400);

Display.AsMarkdown(@"
Created database WebsiteData and container Sales. You can see these new resources by refreshing your resource pane under the Data section.
");


Created database WebsiteData and container Sales. You can see these new resources by refreshing your resource pane under the Data section.


In [2]:
// Creating custom classes to use later
 public class ItemSale
    {
        public string id { get; set; }
        public int CartID { get; set; }
        public ActionType Action { get; set; }
        public string Item { get; set; }
        public double Price { get; set; }
        public string UserName { get; set; }
        public string Country { get; set; }
        public string EventDate { get; set; }
        public int Year { get; set; }
        public double Latitude { get; set; }
        public double Longitude { get; set; }
        public string Address { get; set; }
    } 

 public class ItemSaleWithDiscount
    {
        public string id { get; set; }
        public int CartID { get; set; }
        public ActionType Action { get; set; }
        public string Item { get; set; }
        public double Price { get; set; }
        public string UserName { get; set; }
        public string Country { get; set; }
        public string EventDate { get; set; }
        public int Year { get; set; }
        public double Latitude { get; set; }
        public double Longitude { get; set; }
        public string Address { get; set; }
        public bool IsDiscounted { get; set; }
    }    

 public class ItemSaleTotal
    {
        public string Country { get; set; }
        public double ItemRevenue { get; set; }
        public int ItemsSold { get; set; }
   }
    public enum ActionType
    {
        Viewed,
        Added,
        Purchased
    }

In [3]:
%%upload --databaseName WebsiteData --containerName sales --url https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/websiteData-small.json

Documents successfully uploaded to sales 
Total number of documents imported: 
  Success: 7 
  Failure: 0 
Total time taken : 00:00:00 hours 
Total RUs consumed : 67.97

## Querying your data

Next, we'll write some SQL queries to explore the data in the Sales container. In Azure Cosmos DB SQL API, you can query your data with SQL queries. Azure Cosmos DB SQL is different from the ANSI SQL you might already be familiar with because it's designed for querying JSON documents, as opposed to relational database tables. However, most SQL concepts still apply in Azure Cosmos DB and if you're already familar with SQL, it is easy to learn how to write queries in Azure Cosmos DB.

### The below examples will explore basic query features in Azure Cosmos DB. Here are some things to keep in mind:

    - Azure Cosmos DB indexes all properties by default, so you don't need to manually create indexes. Later on, you can remove unused indexes and add on composite indexes to further improve query performance
    - You don't need to be an expert in database performance to run queries in Azure Cosmos DB. There's virtually no query tuning required and queries automatically use indexes.
    - All queries deduct from your per-second budget of request units (RUs). Queries that are more complex or have higher latency tend to consume more RUs. All queries consume at least 2 RUs.


In [4]:
QueryDefinition queryDefinition = new QueryDefinition("SELECT c.Action, c.Price as ItemRevenue, c.Country, c.Item FROM c");

FeedIterator<ItemSale> queryResultSetIterator = container.GetItemQueryIterator<ItemSale>(queryDefinition);

List<ItemSale> itemSales = new List<ItemSale>();

while (queryResultSetIterator.HasMoreResults)
{
    FeedResponse<ItemSale> currentResultSet = await queryResultSetIterator.ReadNextAsync();
    foreach (ItemSale itemSale in currentResultSet)
    {
        itemSales.Add(itemSale);
    }
}

itemSales

System.Collections.Generic.List`1[Submission#2+ItemSale]

In [5]:
// Using the WHERE clause to filter to a specific country

QueryDefinition queryDefinition = new QueryDefinition("SELECT c.Action, c.Price as ItemRevenue, c.Country, c.Item FROM c WHERE c.Country = 'Iceland'");

FeedIterator<ItemSale> queryResultSetIterator = container.GetItemQueryIterator<ItemSale>(queryDefinition);

List<ItemSale> itemSales = new List<ItemSale>();

while (queryResultSetIterator.HasMoreResults)
{
    FeedResponse<ItemSale> currentResultSet = await queryResultSetIterator.ReadNextAsync();
    foreach (ItemSale itemSale in currentResultSet)
    {
        itemSales.Add(itemSale);
    }
}

itemSales


System.Collections.Generic.List`1[Submission#2+ItemSale]

In [6]:
// Using GROUP BY to obtain the total revenue and quantity sold for each item

QueryDefinition queryDefinition = new QueryDefinition("SELECT SUM(c.Price) as ItemRevenue, Count(1) AS ItemsSold FROM c GROUP BY c.Country");

FeedIterator<ItemSaleTotal> queryResultSetIterator = container.GetItemQueryIterator<ItemSaleTotal>(queryDefinition);

List<ItemSaleTotal> itemSaleTotals = new List<ItemSaleTotal>();

while (queryResultSetIterator.HasMoreResults)
{
    FeedResponse<ItemSaleTotal> currentResultSet = await queryResultSetIterator.ReadNextAsync();
    foreach (ItemSaleTotal itemSaleTotal in currentResultSet)
    {
        itemSaleTotals.Add(itemSaleTotal);
    }
}

itemSaleTotals


System.Collections.Generic.List`1[Submission#2+ItemSaleTotal]

## Adding a new property to a document

Documents within your Azure Cosmos containers can have different structures. Unlike when using a relational database, you don't have to define a schema. Adding a new property to a single document doesn't require any changes at the container-level. 

Here's an example:

In [7]:
// First, we'll read the item that we want to modify
ItemSaleWithDiscount itemSale = await container.ReadItemAsync<ItemSaleWithDiscount>("5ee27756-c380-40c6-adf0-93f48139da1e", new PartitionKey("5ee27756-c380-40c6-adf0-93f48139da1e"));

In [8]:
// Then, we'll modify the ItemSale that we previously read and add the new property "IsDiscounted". We can go ahead and immediately update this in our Sales container, without any changes in settings for that container.
itemSale.IsDiscounted = true;
ItemResponse<ItemSaleWithDiscount> item = await container.UpsertItemAsync<ItemSaleWithDiscount>(itemSale, new PartitionKey(itemSale.id));



## Automatic indexing

This property is instantly available to use in query filters. You don't need to add any new indexes or tune query performance.

In [9]:
// Using the new IsDiscounted property

QueryDefinition queryDefinition = new QueryDefinition("SELECT c.Action, c.Price as ItemRevenue, c.Country, c.Item FROM c WHERE c.IsDiscounted = true");

FeedIterator<ItemSaleWithDiscount> queryResultSetIterator = container.GetItemQueryIterator<ItemSaleWithDiscount>(queryDefinition);

List<ItemSaleWithDiscount> itemSales = new List<ItemSaleWithDiscount>();

while (queryResultSetIterator.HasMoreResults)
{
    FeedResponse<ItemSaleWithDiscount> currentResultSet = await queryResultSetIterator.ReadNextAsync();
    foreach (ItemSaleWithDiscount itemSaleWithDiscount in currentResultSet)
    
        itemSales.Add(itemSaleWithDiscount);
}

itemSales

System.Collections.Generic.List`1[Submission#2+ItemSaleWithDiscount]